In [1]:
import pyrealsense2 as rs
import matplotlib as plt
import cv2
import logging
import numpy as np
import time
import argparse
import glob

In [29]:
ctx = rs.context()
devices = ctx.query_devices()
var = [i.get_info(rs.camera_info.serial_number) for i in devices]

#set up master and slave
dev = devices[0].first_depth_sensor()
dev.set_option(rs.option.inter_cam_sync_mode,1)
dev.get_option(rs.option.inter_cam_sync_mode)

dev2 = devices[1].first_depth_sensor()
dev2.set_option(rs.option.inter_cam_sync_mode,2)
dev2.get_option(rs.option.inter_cam_sync_mode)

#start pipeline
#cam1: configure depth and color streams
pipeline1 = rs.pipeline()
config1 = rs.config()
config1.enable_device(var[0])
config1.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)
config1.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
#config1.enable_record_to_file('master.bag')
print('cam1 enabled')

#cam2: configure depth and color streams 
pipeline2 = rs.pipeline()
config2 = rs.config()
config2.enable_device(var[1])
config2.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)
config2.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
#config2.enable_record_to_file('slave.bag')
print('cam2 enabled')
prof = pipeline1.start(config1)
prof2 = pipeline2.start(config2)
print('cams configued')

cam1 enabled
cam2 enabled
cams configued


In [30]:
#take checkerboard images
try:
    start = time.time()
    mastercount = []
    slavecount = []
    d = 1
    while time.time() - start <100:

        #camera 1
        frames1 = pipeline1.wait_for_frames()
        depthframe1 = frames1.get_depth_frame()
        colframe1 = frames1.get_color_frame()
        if not depthframe1 or not colframe1:
            continue


        #convert images to numpy arrays
        depthim1 = np.asanyarray(depthframe1.get_data())
        colim1 = np.asanyarray(colframe1.get_data())

        #apply color map on depth image, convert to 8bit pixel
        depthcolmap1 = cv2.applyColorMap(cv2.convertScaleAbs(depthim1, alpha=0.5), cv2.COLORMAP_JET)

        #camera 2
        frames2 = pipeline2.wait_for_frames()
        depthframe2 = frames2.get_depth_frame()
        colframe2 = frames2.get_color_frame()
        if not depthframe2 or not colframe2:
            continue


        #check time sync
        #print([depthframe1, depthframe2]) #returns system_time
        #counter = depthframe1.get_frame_metadata(rs.frame_metadata_value.frame_counter)
        #mastercount.append(counter)
        #counter2 = depthframe2.get_frame_metadata(rs.frame_metadata_value.frame_counter)
        #slavecount.append(counter2)
        #timestamp1 = depthframe1.get_timestamp()
        #timestamp2 = depthframe2.get_timestamp()
        #print([counter,counter2, timestamp1, timestamp2])
        #convert images to numpy arrays
        depthim2 = np.asanyarray(depthframe2.get_data())
        colim2 = np.asanyarray(colframe2.get_data())

        #apply color map on depth image, convert to 8bit pixel
        depthcolmap2 = cv2.applyColorMap(cv2.convertScaleAbs(depthim2, alpha=0.5), cv2.COLORMAP_JET)


        #stack all images horizontally
        images = np.hstack((colim1, depthcolmap1, colim2, depthcolmap2))

        #show images from cameras
        cv2.namedWindow('RealSense', cv2.WINDOW_NORMAL)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
        
        #save images and depth map from both cameras by pressing "s"
        ch = cv2.waitKey(25)
        if ch==115:
            filename = "checkerboard/master_%d.jpg"%d
            filename2 = "checkerboard/slave_%d.jpg"%d
            cv2.imwrite(filename, colim1)
            cv2.imwrite(filename2, colim2)
            d+=1
            #cv2.imwrite("depth1.jpg", depthcolmap1)
            #cv2.imwrite("depth2.jpg", depthcolmap2)

finally:
    #stop streaming
    pipeline1.stop()
    pipeline2.stop()
    cv2.destroyAllWindows()
    del pipeline1
    del config1
    del pipeline2
    del config2

In [3]:
#calculate extrinsic and intrinsic params from above checkerboards
#currently for one camera
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
#prepare object points
objp = np.zeros((7*9,3),np.float32)
objp[:,:2] = np.mgrid[0:7,0:9].T.reshape(-1,2)

#arrays to store object points and image points from all images
objpoints = []
imgpoints = []

path = '/home/alissa/IntelRealSense/checkerboard/'
images = glob.glob(path + '/master_*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (7,9),None)
    #if found add object points, image points
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        imgpoints.append(corners2)
        #draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,9), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)
cv2.destroyAllWindows()

In [20]:
#returns camera matrix, distortion coefficients, rotation, and translation vectors
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [24]:
#get extrinsics
cv2.solvePnP(objpoints, corners, mtx, dist)

TypeError: Expected Ptr<cv::UMat> for argument '%s'

In [9]:
print('camera matrix', mtx)
print('distortion coeff', dist)
print('rotation', rvecs)
print('translation', tvecs)

camera matrix [[532.72339941   0.         466.42753055]
 [  0.         510.36751616 243.05093151]
 [  0.           0.           1.        ]]
distortion coeff [[-0.16851522  1.30677432  0.01250941  0.07521741 -4.6680607 ]]
rotation [array([[-0.14758784],
       [ 0.258183  ],
       [-0.93077796]]), array([[-0.16652445],
       [ 0.18601531],
       [-0.20942054]]), array([[-0.10598097],
       [ 0.32237389],
       [-0.28935504]]), array([[0.43968074],
       [0.83892158],
       [1.55071622]]), array([[-0.32789176],
       [ 0.21257754],
       [-0.84488081]]), array([[0.1302787 ],
       [0.17720107],
       [1.05794602]]), array([[0.10906699],
       [0.21261334],
       [1.15867393]]), array([[0.07104742],
       [0.31345124],
       [0.61195049]]), array([[0.04475727],
       [0.20319984],
       [1.09208668]]), array([[0.70246517],
       [0.7135817 ],
       [1.36043029]]), array([[-0.16971438],
       [-0.17936405],
       [ 1.13941748]]), array([[-0.08312169],
       [-0.28403

In [11]:
pipeline = rs.pipeline()
pipeline.cong
profile = pipeline.get_active_profile()

RuntimeError: get_active_profile() can only be called between a start() and a following stop()